<a href="https://colab.research.google.com/github/sipocz/makerere/blob/main/zindi_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Augmentor

In [2]:
url="https://api.zindi.africa/v1/competitions/makerere-fall-armyworm-crop-challenge/files/Images.zip"
url_train="https://api.zindi.africa/v1/competitions/makerere-fall-armyworm-crop-challenge/files/Train.csv"
url_test="https://api.zindi.africa/v1/competitions/makerere-fall-armyworm-crop-challenge/files/Test.csv"
url_sample_s="https://api.zindi.africa/v1/competitions/makerere-fall-armyworm-crop-challenge/files/SampleSubmission.csv"
auth={'auth_token': "KXxbUBzf4GQ6pqkNGC8zJt8B"}


In [3]:
# Function to download data
def zindi_data_downloader(data_url, auth, file_name):
    import requests
    from tqdm.auto import tqdm
    # Get the competition data
    x = requests.post(data_url, data = auth,stream=True)
    target_path = 'data.zip'

    handle = open(file_name, "wb")
    for chunk in x.iter_content(chunk_size=512):
        if chunk:  # filter out keep-alive new chunks
            handle.write(chunk)
    handle.close()

In [4]:
zindi_data_downloader(url,auth,"Images.zip")

In [5]:
!mkdir pic

In [6]:
!unzip -q  /content/Images.zip -d /content/pic

In [7]:
!rm *.zip

In [8]:
zindi_data_downloader(url_train,auth,"Train.csv")
zindi_data_downloader(url_test,auth,"Test.csv")
zindi_data_downloader(url_sample_s,auth,"SampleSubmission.csv")



In [9]:
import pandas as pd

In [10]:
df=pd.read_csv("Train.csv")
df["fullpath"]="/content/pic/"+df["Image_id"]
o=["zero" if i==0 else "one" for i in df["Label"].values ]
df["cat"]=o

In [12]:
df.head()

,Image_id,Label,fullpath,cat
0,id_02amazy34fgh2.jpg,1,/content/pic/id_02amazy34fgh2.jpg,one
1,id_02mh3w48pmyc9.jpg,0,/content/pic/id_02mh3w48pmyc9.jpg,zero
2,id_02rpb463h9d3w.jpg,0,/content/pic/id_02rpb463h9d3w.jpg,zero
3,id_02wc3jeeao8ol.jpg,1,/content/pic/id_02wc3jeeao8ol.jpg,one
4,id_03t2hapb8wz8p.jpg,1,/content/pic/id_03t2hapb8wz8p.jpg,one


In [13]:
ids=df["Image_id"].to_list()
labels=df["Label"].to_list()

In [14]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D,Dense,Dropout
from tensorflow.keras.models  import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [15]:
from numpy.ma.core import resize
x_px=500
y_px=500
bpc=3
data_path="/content/pic/"
batch_size=10
train=ImageDataGenerator(horizontal_flip=True, vertical_flip=True,validation_split=0.1,rescale=1./255,
                         shear_range = 0.15,zoom_range = 0.15,
                         width_shift_range = 0.15,
                         height_shift_range = 0.15,)

test=ImageDataGenerator(rescale=1./255,validation_split=0.1)




print(f"X: {x_px}, Y: {y_px}")
train_generator=train.flow_from_dataframe(df,x_col="fullpath",y_col="cat",target_size=(x_px,y_px),batch_size=batch_size,class_mode='binary',subset='training',color_mode="rgb")
test_generator = test.flow_from_dataframe(df,x_col="fullpath",y_col="cat",target_size=(x_px,y_px),batch_size=batch_size,class_mode='binary',subset='validation',color_mode="rgb")


X: 500, Y: 500
Found 1458 validated image filenames belonging to 2 classes.
Found 161 validated image filenames belonging to 2 classes.


In [16]:
# Build Model
# Importáld a megfelelő rétegeket
from tensorflow.keras.layers import Input,Dense,Embedding,LSTM,TimeDistributed, Flatten, Bidirectional, Conv2D, Dropout,Convolution2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D, Rescaling,RandomFlip,RandomRotation, Reshape,BatchNormalization,MaxPooling3D

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adadelta,Adam,SGD,Nadam,Ftrl,SGD,Adamax
from tensorflow.keras.losses import sparse_categorical_crossentropy,categorical_crossentropy, mean_squared_error
from tensorflow.keras import backend as K
from keras.regularizers import l1
from keras.initializers import RandomNormal,Zeros,GlorotUniform

K.clear_session()



inputs= Input(shape=(x_px,y_px,bpc))
e = Conv2D(128, (2, 2), activation='relu')(inputs)
e = MaxPooling2D((2, 2))(e)

e = Conv2D(64, (2, 2), activation='relu')(e)
e = MaxPooling2D((2, 2))(e)
e = Conv2D(128, (2, 2), activation='relu')(e)
e = MaxPooling2D((2, 2))(e)



l = Flatten()(e)
        #l = Dense(1920, activation='sigmoid')(l)
         # overfitting compensation ??
        #l=Dropout(0.7)(l)
        #l = Dense(74, activation='sigmoid')(l)
    #l=Dropout(0.6)(l)
    #l = Dense(56, activation='sigmoid')(l)
l=Dropout(0.95)(l)


decoded = Dense(2, activation='softmax')(l)


convmodel = Model(inputs, decoded)


In [17]:

cce = tf.keras.losses.SparseCategoricalCrossentropy()
acc = tf.keras.metrics.SparseCategoricalAccuracy()
optimizer=Adam()
convmodel.compile(loss=cce, optimizer=optimizer, metrics=[acc])
convmodel.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 500, 500, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 499, 499, 128)     1664      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 249, 249, 128)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 248, 248, 64)      32832     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 124, 124, 64)     0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 123, 123, 128)     32896 

In [ ]:
  history = convmodel.fit(train_generator, epochs=100,  validation_data=test_generator,batch_size=1)

Epoch 1/100
146/146 [==============================] - 137s 857ms/step - loss: 0.7356 - sparse_categorical_accuracy: 0.4897 - val_loss: 0.6847 - val_sparse_categorical_accuracy: 0.5093
Epoch 2/100
146/146 [==============================] - 125s 856ms/step - loss: 0.6856 - sparse_categorical_accuracy: 0.5576 - val_loss: 0.6802 - val_sparse_categorical_accuracy: 0.6708
Epoch 3/100
146/146 [==============================] - 123s 841ms/step - loss: 0.6717 - sparse_categorical_accuracy: 0.5898 - val_loss: 0.9595 - val_sparse_categorical_accuracy: 0.5404
Epoch 4/100
146/146 [==============================] - 125s 852ms/step - loss: 0.6787 - sparse_categorical_accuracy: 0.5864 - val_loss: 0.6766 - val_sparse_categorical_accuracy: 0.5590
Epoch 5/100
146/146 [==============================] - 127s 868ms/step - loss: 0.6588 - sparse_categorical_accuracy: 0.6241 - val_loss: 0.5951 - val_sparse_categorical_accuracy: 0.7516
Epoch 6/100
146/146 [==============================] - 126s 865ms/step - lo